In [8]:
import numpy as np
import pandas as pd
import os
import pickle
from tqdm import tqdm

# Get the size of the file for the progress bar
file_path = "E:/fab-data/simplified_tournament_data_new_pairings.pkl"
file_size = os.path.getsize(file_path)

# A list to hold all tournament data objects
all_tournament_data = []

# Open the pickle file for reading
with open(file_path, "rb") as f:
    with tqdm(total=file_size, unit='B', unit_scale=True, desc='Loading data') as pbar:
        while True:
            try:
                # Tell the progress bar how much data has been read so far
                pos_before = f.tell()
                
                # Load the next tournament data object from the file
                tournament_data = pickle.load(f)
                
                # Update the progress bar with the number of bytes read
                pos_after = f.tell()
                pbar.update(pos_after - pos_before)
                
                all_tournament_data.append(tournament_data)
            except EOFError:
                # No More data
                break
print(len(all_tournament_data))

Loading data: 100%|██████████| 1.46G/1.46G [02:43<00:00, 8.91MB/s]

100000


# Loading Player and Deck data

In [9]:
# Constants
default_skill_value = 0 # value for a players skill if no player_skill is provided on player creation
skill_value_modifier = 0.03
total_of_decks = 11 # total amount of decks
player_num = 512 # total number of players in a tournament
num_rounds = 14 # total number of rounds in a tournament



# Player Tournament and classes
import numpy as np
import pandas as pd
from threading import Lock
from itertools import combinations
import re

class Deck:
    def __init__(self, name):
        self.name = name
        self.matchup_spread = {}

    def set_matchup_win_prob(self, opponent_deck_name, win_prob):
        self.matchup_spread[opponent_deck_name] = win_prob

    def get_matchup_win_prob(self, opponent_deck_name):
        return self.matchup_spread.get(opponent_deck_name, 0.5)
    
    def __str__(self):
        return self.name
    
    def __reduce__(self):
        # Return the class itself, arguments, and the state
        return (self.__class__, (self.name,), {'name': self.name, 'matchup_spread': self.matchup_spread})
    
    def __setstate__(self, state):
        # Set the object's state from the given state dictionary
        self.__dict__.update(state)

class DeckManager:
    def __init__(self):
        self.decks = {}

    def add_deck(self, deck):
        self.decks[deck.name] = deck

    def generate_win_probabilities(self, deck_names):
        num_decks = len(deck_names)
        for i in range(num_decks):
            for j in range(i + 1, num_decks):
                win_prob = np.clip(np.random.normal(0.5, 0.15), 0, 1)
                self.decks[deck_names[i]].set_matchup_win_prob(deck_names[j], win_prob)
                self.decks[deck_names[j]].set_matchup_win_prob(deck_names[i], 1 - win_prob)

    def get_win_prob_matrix(self):
        deck_names = list(self.decks.keys())
        num_decks = len(deck_names)
        matrix = np.zeros((num_decks, num_decks))
        for i in range(num_decks):
            for j in range(num_decks):
                matrix[i, j] = self.decks[deck_names[i]].get_matchup_win_prob(deck_names[j])
        return matrix

    def get_win_prob_dataframe(self):
        deck_names = list(self.decks.keys())
        num_decks = len(deck_names)
        matrix = np.zeros((num_decks, num_decks))
        for i in range(num_decks):
            for j in range(num_decks):
                matrix[i, j] = self.decks[deck_names[i]].get_matchup_win_prob(deck_names[j])
        df = pd.DataFrame(matrix, index=deck_names, columns=deck_names)
        return df

    def load_win_probabilities_from_csv(self, file_path):
        # Read the Excel file
        df = pd.read_csv(file_path, header=0, index_col=0)
        
        # Drop the Representation column if it exists
        if 'Representation' in df.columns:
            df = df.drop(columns=['Representation'])
        
        # Ensure decks are added to the manager
        deck_names = df.index.to_list()
        for deck_name in deck_names:
            if deck_name not in self.decks:
                self.add_deck(Deck(deck_name))
        
        # Populate the matchup probabilities
        for i, row in df.iterrows():
            for j, value in row.items():
                self.decks[i].set_matchup_win_prob(j, value)
    
    def __reduce__(self):
        # Return the class itself, no arguments, and the state
        return (self.__class__, (), {'decks': self.decks})
    
    def __setstate__(self, state):
        # Set the object's state from the given state dictionary
        self.__dict__.update(state)

class Player:
    def __init__(self, player_id, alias, player_skill=default_skill_value, deck=None):
        self.id = player_id
        self.alias = alias
        self.skill = player_skill
        self.deck = deck
        self.wins = 0
        self.losses = 0
        self.history = []  # List to store match results
        self.history_lock = Lock()
    
    def __reduce__(self):
        # The object's state is returned as a tuple:
        # (callable, arguments_to_callable, additional_state)
        # Lock object is not pickled, so we're not including it in the state.
        return (self.__class__, (self.id, self.alias, self.skill, self.deck), {'wins': self.wins, 'losses': self.losses, 'history': self.history})
    
    def __setstate__(self, state):
        self.wins = state.get('wins', 0)
        self.losses = state.get('losses', 0)
        self.history = state.get('history', [])
        self.history_lock = Lock()  # Initialize a new Lock object after unpickling

    def set_deck(self, deck):
        """Set the deck for the player."""
        self.deck = deck

    def get_deck(self):
        """Retrieve the player's deck."""
        return self.deck

    def record_match(self, tournament, tournament_round, opponent, result):
        with self.history_lock:
            self.history.append([tournament.name, tournament_round, str(opponent.id) + opponent.alias, result])
    
    def get_tournament_vector(self):
        # Assuming result is the last element in the history entry
        return np.array([1 if entry[-1] == "W" else -1 for entry in self.history])
    
    def get_tournament_history(self, tournament):
        tournament_results = [history for history in self.history if history[0] == tournament.name]
        return tournament_results

    def get_tournament_w_l(self, tournament):
        tournament_results = [history for history in self.history if history[0] == tournament.name]
        wins = 0
        losses = 0
        for match in tournament_results:
            result = match[-1]
            if result == "W":
                wins += 1
            elif result == "L":
                losses += 1
        return (wins, losses)

    def get_tournament_standing(self):
        """
        Calculate the player's tournament standing based on their match record.
        
        :param player_record: A list of 'W' and 'L' indicating wins and losses.
        :return: A tuple containing total match points and cumulative tiebreaker score.
        """
        player_record = [entry[-1] for entry in self.history]
        total_rounds = num_rounds
        match_points = [1 if record == "W" else 0 for record in player_record]
        total_points = 0
        for match in match_points:
            total_points += match
        ctb_numerator = sum((1 if result == 'W' else 0) * (0.25 ** (total_rounds - round_number))
                            for round_number, result in enumerate(player_record, 1))
        ctb_denominator = sum(round_number * (0.25 ** (total_rounds - round_number))
                            for round_number in range(1, total_rounds + 1))
        ctb = ctb_numerator / ctb_denominator if ctb_denominator else 0
        
        return total_points, ctb

    def __str__(self):
        return f"[{self.id}] {self.alias} ({self.get_deck().name}) - Skill Level: {self.skill}"

    def __repr__(self):
        return f"[{self.id}] {self.alias} ({self.get_deck().name}) - Skill Level: {self.skill}"


class Tournament:
    def __init__(self, name, player_aliases, win_prob_matrix):
        self.name = name
        self.players = player_aliases
        self.win_prob_matrix = win_prob_matrix
        self.rounds_played = 0
        self.current_draft_standings = {player: [] for player in self.players}  # Use lists to hold match history

    def __reduce__(self):
        # Assuming win_prob_matrix is a numpy array, which is pickleable.
        # players list is also assumed to be pickleable.
        return (self.__class__, (self.name, self.players, self.win_prob_matrix), {'rounds_played': self.rounds_played})

    def __setstate__(self, state):
        self.rounds_played = state.get('rounds_played', 0)
        
    def get_tournament_standings(self):
        players_standings = [(player, player.get_tournament_standing()) for player in self.players]
        
        # Sort players based on match points and CTB (tiebreaker), descending order
        sorted_players = [standing[0] for standing in sorted(players_standings, key=lambda x: (x[1][0] + x[1][1]), reverse=True)]  # return players and their standings
        return sorted_players
        
    def inspect_player(self, id):
        player_trajectory = {}
        player_history = self.players[id].get_tournament_history(self)
        
        opponents = []
        win_loss = []
        round_number = []
        for match in player_history:
            opp_id = int(re.findall(r'\d+', match[-2])[0])
            opponents.append(self.players[opp_id])
            win_loss.append(match[-1])
            round_number.append(match[-3])
        
        opp_id = []
        opp_name = []
        opp_deck = []
        opp_skill = []
        opp_traj = []
        for opp in opponents:
            opp_id.append(opp.id)
            opp_name.append(opp.alias)
            opp_deck.append(opp.deck.name)
            opp_skill.append(opp.skill)
            win_loss_opp = []
            for match in self.players[opp.id].get_tournament_history(self):
                win_loss_opp.append(match[-1])
            opp_traj.append(win_loss_opp)
        
        player_trajectory['opp_id'] = opp_id
        player_trajectory['opp_name'] = opp_name
        player_trajectory['opp_deck'] = opp_deck
        player_trajectory['opp_skill'] = opp_skill
        player_trajectory['win_loss'] = win_loss
        player_trajectory['opp_win_loss'] = opp_traj
        
        player_df = pd.DataFrame(player_trajectory)
        return player_df

    def swiss_pairings(self):
        """
        Pair players based on their standings.
        
        :param players_standings: A list of tuples, each containing a player ID, their match points, and CTB.
        :return: A list of tuples, each representing a paired match.
        """
        
        player_standings = self.get_tournament_standings()
        
        # Pair players
        pairings = []
        while player_standings:
            # Pop the highest-ranked player
            highest_player = player_standings.pop(0)
            
            # Find the next highest-ranked player who is not yet paired
            for i, opponent in enumerate(player_standings):
                # Assuming no byes, and players can be paired directly
                pairings.append((highest_player, opponent))
                player_standings.pop(i)
                break
        
        return pairings

    def create_pods(self):
        # Create pods based on current standings
        players_standings = self.get_tournament_standings()
        
        return [players_standings[i:i+8] for i in range(0, len(players_standings), 8)]

    def draft_swiss_pairings(self, pod):
        # Sort players by their win records first
        sorted_pod_players = sorted(pod, key=lambda x: self.current_draft_standings[x], reverse=True)
        
        # Pair players
        pairings = []
        while sorted_pod_players:
            # Pop the highest-ranked player
            highest_player = sorted_pod_players.pop(0)
            
            # Find the next highest-ranked player who is not yet paired
            for i, opponent in enumerate(sorted_pod_players):
                # Assuming no byes, and players can be paired directly
                pairings.append((highest_player, opponent))
                sorted_pod_players.pop(i)
                break
            
        return pairings

    def simulate_round(self):
        pairings = self.swiss_pairings()
        self.rounds_played += 1
        for p1, p2 in pairings:
            index_A = self.players.index(p1)
            index_B = self.players.index(p2)
            r = np.random.rand()
            if r < self.win_prob_matrix[index_A, index_B]:
                p1.record_match(self, self.rounds_played, p2, 'W')
                p2.record_match(self, self.rounds_played, p1, 'L')
            else:
                p1.record_match(self, self.rounds_played, p2, 'L')
                p2.record_match(self, self.rounds_played, p1, 'W')

    # round simulation
    def simulate_constructed_rounds(self, n_rounds):
        for _ in range(n_rounds):
            self.simulate_round()
    
    def simulate_draft_round(self, pod):
        pairings = self.draft_swiss_pairings(pod)
        for p1, p2 in pairings:
            p1_win_percentage = 0.5 + skill_value_modifier*(p1.skill - p2.skill)
            r = np.random.rand()
            if r < p1_win_percentage:
                p1.record_match(self, self.rounds_played, p2, 'W')
                p2.record_match(self, self.rounds_played, p1, 'L')
                self.current_draft_standings[p1].append('W')  # Append 'W' to the list for player 1
                self.current_draft_standings[p2].append('L')  # Append 'L' to the list for player 2
            else:
                p1.record_match(self, self.rounds_played, p2, 'L')
                p2.record_match(self, self.rounds_played, p1, 'W')
                self.current_draft_standings[p1].append('L')  # Append 'W' to the list for player 2
                self.current_draft_standings[p2].append('W')  # Append 'L' to the list for player 1
    
    def simulate_draft_rounds(self, n_rounds):
        self.current_draft_standings = {player: [] for player in self.players}  # Use lists to hold match history
        pods = self.create_pods()
        for _ in range(n_rounds):
            self.rounds_played += 1
            for pod in pods:
                self.simulate_draft_round(pod)
        

    def simulate_tournament(self):
        # Format for worlds (is it really 4 cc - 3 draft - 3 draft - 4 cc and cut???)
        self.simulate_constructed_rounds(4)
        self.simulate_draft_rounds(3)
        self.simulate_draft_rounds(3)
        self.simulate_constructed_rounds(4)

    def display_results(self):
        for player in self.players:
            print(f"{player.alias}: {player.get_tournament_history(self)}")
    
    def display_standings(self):
        players_standings = [(player, player.get_tournament_standing()) for player in self.players]
        
        # Sort players based on match points and CTB (tiebreaker), descending order
        sorted_players = sorted(players_standings, key=lambda x: (x[1][0] + x[1][1]), reverse=True)  # return players and their standings
        for player in sorted_players:
            points, ctb = player.get_tournament_standing(self)
            print(f"{player}: {points}-{ctb}")
    
    def get_player_by_id(self, player_id):
        """
        Return a player object by its ID.
        """
        for player in self.players:
            if player.id == player_id:
                return player
        return None

    def get_top_n(self, n):
        """
        Return the top n players based on their score and common tiebreakers.
        """
        # First sort by the primary score based on wins and similarity score.
        players_standings = self.get_tournament_standings()

        # # RULES FOR TIEBREAKER. CANT FIX, TOO HARD
        # # Now sort by the common tiebreakers.
        # final_sorted_players = sorted(
        #     primary_sorted_players,
        #     key=lambda player: (
        #         player.get_tournament_score(self),  # Primary score
        #         player.get_opponents_match_win_percentage(self),  # OMW%
        #         player.get_game_win_percentage(self),  # GW%
        #         player.get_opponents_game_win_percentage(self)  # OGW%
        #     ),
        #     reverse=True
        # )

        return players_standings[:n]
    
    def get_win_prob_dataframe(self):
        """Retrieve the win probability for each player matrix as a DataFrame."""
        player_names = [str(player.id) + player.alias for player in self.players]
        
        # Convert to DataFrame for better visualization
        df = pd.DataFrame(self.win_prob_matrix, index=player_names, columns=player_names)
        return df
    
    def __str__(self):
        return f"{self.name}"

    def __repr__(self):
        return f"{self.name}"

In [10]:
with open("deck_manager.pkl", "rb") as f:
    deck_manager = pickle.load(f)
with open("players.pkl", "rb") as f:
    players = pickle.load(f)

# Analysis

In [11]:
import pandas as pd

def update_player_tournament_placements(player_placements, tournament):
    top_64 = tournament['top_64']
    for placement, player_id in enumerate(top_64, start=1):
        # Initialize the player's record if not present
        if player_id not in player_placements:
            player_placements[player_id] = {'top_8': 0, 'top_16': 0, 'top_32': 0, 'top_64': 0}
        
        # Update top placements
        if placement <= 8:
            player_placements[player_id]['top_8'] += 1
        elif placement <= 16:
            player_placements[player_id]['top_16'] += 1
        elif placement <= 32:
            player_placements[player_id]['top_32'] += 1
        elif placement <= 64:
            player_placements[player_id]['top_64'] += 1

def create_dataframe(players, player_placements):
    # Prepare the data for the DataFrame
    data = []
    for player in players:
        # Include only players who have placement records
        if player.id in player_placements:
            data.append({
                'player_id': player.id,
                'alias': player.alias,
                'skill': round(player.skill, 3),
                'deck_name': player.deck.name,
                'top_8': player_placements[player.id]['top_8'],
                'top_16': player_placements[player.id]['top_16'],
                'top_32': player_placements[player.id]['top_32'],
                'top_64': player_placements[player.id]['top_64'],
            })

    # Create the DataFrame
    df = pd.DataFrame(data)
    return df


In [15]:
player_placements = {}
for tournament in all_tournament_data:  # Assuming 'tournaments' is your list of tournament dicts
    update_player_tournament_placements(player_placements, tournament)

# Assuming 'players' is your list of player objects
df = create_dataframe(players, player_placements)

# Export the DataFrame to a CSV file
df.to_csv('player_placements.csv', index=False)


df_sorted = df.sort_values(by='top_8', ascending=False)
df_sorted

,player_id,alias,skill,deck_name,top_8,top_16,top_32,top_64
333,333,Pauline,2.553,Katsu,6060,4768,7174,12185
252,252,Ruby,1.853,Dromai,5123,4297,7155,12032
229,229,Jean,1.412,Dash,5038,4541,6891,11981
447,447,Ernest,1.657,Dromai,5009,4412,6946,11927
130,130,Keisha,2.200,Katsu,4837,3716,6679,10963
...,...,...,...,...,...,...,...,...
1,1,Orville,-1.022,Boltyn,130,260,628,1680
228,228,Amanda,-1.556,Uzuri,111,210,602,1469
376,376,George,-1.308,Uzuri,95,131,430,1342
328,328,Susan,-2.240,Uzuri,86,136,371,1076


In [13]:
# Constants
TOTAL_TOURNAMENTS = len(all_tournament_data)

# Group by 'deck_name' and aggregate the data
deck_grouped = df.groupby('deck_name').agg({
    'skill': ['mean', 'max'],
    'top_8': 'sum',
    'top_16': 'sum',
    'top_32': 'sum',
    'top_64': 'sum',
}).reset_index()

# Flatten the MultiIndex in columns
deck_grouped.columns = ['_'.join(col).strip() if col[1] else col[0] for col in deck_grouped.columns.values]

# Rename the columns
deck_grouped.rename(columns={
    'skill_mean': 'average_skill',
    'skill_max': 'highest_skill',
    'top_8_sum': 'total_top_8',
    'top_16_sum': 'total_top_16',
    'top_32_sum': 'total_top_32',
    'top_64_sum': 'total_top_64',
}, inplace=True)

# Calculate the frequency of top placements for each deck
deck_grouped['frequency_top_8'] = deck_grouped['total_top_8'] / TOTAL_TOURNAMENTS
deck_grouped['frequency_top_16'] = deck_grouped['total_top_16'] / TOTAL_TOURNAMENTS
deck_grouped['frequency_top_32'] = deck_grouped['total_top_32'] / TOTAL_TOURNAMENTS
deck_grouped['frequency_top_64'] = deck_grouped['total_top_64'] / TOTAL_TOURNAMENTS

# Calculate the representation of each deck in top placements
deck_grouped['representation_top_8'] = deck_grouped['total_top_8'] / (8 * TOTAL_TOURNAMENTS)
deck_grouped['representation_top_16'] = deck_grouped['total_top_16'] / (16 * TOTAL_TOURNAMENTS)
deck_grouped['representation_top_32'] = deck_grouped['total_top_32'] / (32 * TOTAL_TOURNAMENTS)
deck_grouped['representation_top_64'] = deck_grouped['total_top_64'] / (64 * TOTAL_TOURNAMENTS)

# Optionally, sort the DataFrame based on a relevant column
deck_grouped.sort_values(by='representation_top_8', ascending=False, inplace=True)

# Export the DataFrame to a CSV file
deck_grouped.to_csv('deck_aggregated_data_with_representations.csv', index=False)
deck_grouped

,deck_name,average_skill,highest_skill,total_top_8,total_top_16,total_top_32,total_top_64,frequency_top_8,frequency_top_16,frequency_top_32,frequency_top_64,representation_top_8,representation_top_16,representation_top_32,representation_top_64
5,Dromai,-0.019939,1.920,245650,232947,465679,895915,2.45650,2.32947,4.65679,8.95915,0.307063,0.145592,0.145525,0.139987
3,Dash,-0.082710,1.944,124757,118161,236907,456584,1.24757,1.18161,2.36907,4.56584,0.155946,0.073851,0.074033,0.071341
8,Katsu,0.038019,2.553,104427,96492,189334,363605,1.04427,0.96492,1.89334,3.63605,0.130534,0.060307,0.059167,0.056813
2,Bravo,-0.046435,1.865,84654,82049,163715,320747,0.84654,0.82049,1.63715,3.20747,0.105817,0.051281,0.051161,0.050117
6,Fai,0.096167,1.830,64843,68684,136589,282151,0.64843,0.68684,1.36589,2.82151,0.081054,0.042928,0.042684,0.044086
7,Iyslander,-0.030522,1.887,52837,60388,120004,254943,0.52837,0.60388,1.20004,2.54943,0.066046,0.037742,0.037501,0.039835
9,Rhinar,-0.215963,2.010,45911,43895,88896,173738,0.45911,0.43895,0.88896,1.73738,0.057389,0.027434,0.027780,0.027147
0,Azalea,0.057703,2.308,30416,35775,73393,161765,0.30416,0.35775,0.73393,1.61765,0.038020,0.022359,0.022935,0.025276
4,Dorinthea,-0.037500,1.854,18222,22740,44724,99458,0.18222,0.22740,0.44724,0.99458,0.022777,0.014212,0.013976,0.015540
10,Uzuri,0.113579,2.553,17567,24934,52158,125106,0.17567,0.24934,0.52158,1.25106,0.021959,0.015584,0.016299,0.019548
